**Homework 3**
## Install Necessary Tools and libraries

In [1]:
!pip install -qqq tiktoken
!pip install -qqq pinecone-client
!pip install -qqq pypdf
!pip install -qqq openai
!pip install -qqq groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.1 MB/s eta 0:00:00


Import the api key from OpenAI, Pinecone, & Groq

In [3]:
from google.colab import userdata
from groq import Groq
OPENAI_API_KEY=userdata.get('openai-key')
PINECONE_API_KEY=userdata.get('pinecone-key')
GROQ_API_KEY=userdata.get('groq-key')
!pip install -qqq langchain_community
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00


In [4]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

## Load the document

In [5]:
!pip install PyPDF2
from PyPDF2 import PdfReader
import os

def read_doc(directory: str) -> list[str]:
    # Function to read the PDFs from a directory.

    # Args:
    # directory (str): The path of the directory where the PDFs are stored.

    # Returns:
    # list[str]: A list of text in the PDFs.

    page_contents = []

    # List all PDF files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            filepath = os.path.join(directory, filename)

            # Open the PDF file
            reader = PdfReader(filepath)
            num_pages = len(reader.pages)

            # Extract text from each page
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                page_contents.append(page.extract_text())

    return page_contents

# Call the function
full_document = read_doc("/content")
print(len(full_document))  # This should print the number of pages read


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00
143


##Embeddings for only 15 pages





In [9]:
!pip install -qqq langchain_openai
from langchain_openai import OpenAIEmbeddings

# Initialize the embedding model
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")

def generate_embeddings(documents: list[str]) -> list[list[float]]:
    # Generate embeddings for a list of documents.
    # Args:
    # documents (list[str]): A list of document objects, each containing a 'page_content' attribute.
    # Returns:
    # list[list[float]]: A list containing a list of embeddings corresponding to the documents.

    embedded = [embed_model.embed_documents([doc]) for doc in documents]
    return embedded

# Extract the first 15 pages from full_document
selected_documents = full_document[:15]

# Run the function
chunked_document_embeddings = generate_embeddings(documents=selected_documents)

# Check the dimension of embedding model
print(len(chunked_document_embeddings))

15


In [11]:
# create unique IDs
ids = [str(x) for x in range(0,len(chunked_document_embeddings))]

def combine_vector_and_text(
    documents: list[any], doc_embeddings: list[list[float]]) -> list[dict[str, any]]:
    """
    Process a list of documents along with their embeddings.

    Args:
    - documents (List[Any]): A list of documents (strings or other types).
    - doc_embeddings (List[List[float]]): A list of embeddings corresponding to the documents.

    Returns:
    - data_with_metadata (List[Dict[str, Any]]): A list of dictionaries, each containing an ID, embedding values, and metadata.
    """
    data_with_metadata = []

    for id,doc_text, embedding in zip(ids,documents, doc_embeddings):
        # Convert doc_text to string if it's not already a string
        if not isinstance(doc_text, str):
            doc_text = str(doc_text)

        # Generate a unique ID based on the text content
        doc_id = id

        # Create a data item dictionary
        data_item = {
            "id": doc_id,
            "values": embedding[0],
            "metadata": {"text": doc_text},  # Include the text as metadata
        }

        # Append the data item to the list
        data_with_metadata.append(data_item)

    return data_with_metadata


# Call the function
all_meta_data = combine_vector_and_text(full_document, chunked_document_embeddings)

## Set up the Pinecone Indexes

In [12]:
from pinecone import Pinecone,ServerlessSpec
pinecone = Pinecone()

INDEX_NAME="rag-pdf"
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=3072, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [13]:
print(index)

In [14]:
index.upsert(all_meta_data)

{'upserted_count': 15}

Now, query the pincone indexes

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Generate embeddings for the query
def query_pinecone_index(query_embeddings: list, top_k=3, include_metadata: bool = True) -> dict:
    """
    Query a Pinecone index.

    Args:
    - query_embeddings (list): List of query embeddings.
    - top_k (int): Number of nearest neighbors to retrieve (default: 3).
    - include_metadata (bool): Whether to include metadata in the query response (default: True).

    Returns:
    - query_response (dict): Query response containing nearest neighbors.
    """
    query_response = index.query(
        vector=query_embeddings, top_k=top_k, include_metadata=include_metadata
    )
    return query_response

## Red Teaming with Thresholds
To compare the similarity of the query in the context of provided document, I have set a threshold for the similarity score. The similarity score is returned as a result of querying the Pinecone indexes after receiving a query. The threshold was determined after observing the score values from several queries that were both in-context and out-of-context.

If the similarity score exceeds the threshold, the application identifies the query as valid. Then, it will look for the most similar chunks within the document.

Implementing this technique, will help ward off certain kinds of off-topic queries that have no relation to the document provided.


In [18]:
def query_with_similarity_threshold(query: str, threshold: float = 0.4) -> str:
    """
    Handle the query by determining if it is on-topic or off-topic using red teaming.

    Args:
    - query (str): User's query.
    - threshold (float): Similarity threshold to classify questions as on-topic or off-topic.

    Returns:
    - str: Response to the query.
    """
    # Generate embedding for the query
    query_embedding = embed_model.embed_query(query)

    # Query the Pinecone index
    query_response = query_pinecone_index(query_embeddings=query_embedding)

    # Extract the similarities and metadata
    similarities = [result['score'] for result in query_response['matches']]
    documents = [result['metadata']['text'] for result in query_response['matches']]

    # Check if the query is on-topic
    max_similarity = max(similarities)
    if max_similarity < threshold:
        return "I'm sorry, the query is out of context in relation to the document provide!"

    # Find the most relevant document chunk
    most_relevant_index = np.argmax(similarities)
    return f"{documents[most_relevant_index]}"


Now, invoke the LLM to provide the response after determining that the query is valid query.

## Read Teaming Validation
Ouery 1 - Out of context

In [31]:

query = "Is Taylor Swift better than Katy Perry?"
response = query_with_similarity_threshold(query)

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

LLM1 = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
LLM2 = ChatOpenAI(temperature=0, model_name="gpt-4")
LLM3 = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Extract the text from the dictionary before passing it to the LLM
prompt = f"{response} Using only the provided information, give me a summarized answer on {query}"

def query_response(prompt: str, LLM) -> str:
    """This function returns a better response using LLM
    Args:
        prompt (str): The prompt template

    Returns:
        str: The actual response returned by the LLM
    """
    answer = LLM.invoke(prompt)
    return answer.content

# Response from the red teaming technique
print(f"\n RAG answer: \n {response}")

# Call three LLM functions
final_answer = query_response(prompt=prompt, LLM=LLM1)
print(f"\n Response of gpt-3.5-turbo: \n {final_answer}")

final_answer = query_response(prompt=prompt, LLM=LLM2)
print(f"\n Response of GPT-4: \n {final_answer}")

final_answer = query_response(prompt=prompt, LLM=LLM3)
print(f"\n Response of GPT-4omni: \n {final_answer}")


 RAG answer: 
 I'm sorry, the query is out of context in relation to the document provide!

 Response of gpt-3.5-turbo: 
 It is subjective and ultimately up to personal preference as to whether Taylor Swift is better than Katy Perry. Both artists have achieved great success in the music industry and have their own unique styles and fan bases.

 Response of GPT-4: 
 The document does not provide information on whether Taylor Swift is better than Katy Perry.

 Response of GPT-4omni: 
 The provided information does not include any specific details or criteria to compare Taylor Swift and Katy Perry. Therefore, I cannot provide a summarized answer on whether Taylor Swift is better than Katy Perry based on the given context.


**Observations:**
In the first query, I provided a totally different question that isn't related to the document. The response after the RAG search was as expected. The similarity score didn't pass the threshold and provided a fall-back response, which was expected.

In case of 3 different GPT models, the results were mixed. 3.5-turbo didn't seem to be able to handle off-topic queries as good as GPT-4 and GPT-4o. It still relied on its training data, completely ignoring the prompt which explicitly says, "using only the provided information".

##Query2

In [32]:

query = "Is General Motors building new Cadillac BLS at Saab factory in Sweden?"
response = query_with_similarity_threshold(query)

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

LLM1 = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
LLM2 = ChatOpenAI(temperature=0, model_name="gpt-4")
LLM3 = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Extract the text from the dictionary before passing it to the LLM
prompt = f"{response} Using only the provided information, give me a summarized answer on {query}"

def query_response(prompt: str, LLM) -> str:
    """This function returns a better response using LLM
    Args:
        prompt (str): The prompt template

    Returns:
        str: The actual response returned by the LLM
    """
    answer = LLM.invoke(prompt)
    return answer.content

# Response from the red teaming technique
print(f"\n RAG answer: \n {response}")

# Call three LLM functions
final_answer = query_response(prompt=prompt, LLM=LLM1)
print(f"\n Response of gpt-3.5-turbo: \n {final_answer}")

final_answer = query_response(prompt=prompt, LLM=LLM2)
print(f"\n Response of GPT-4: \n {final_answer}")

final_answer = query_response(prompt=prompt, LLM=LLM3)
print(f"\n Response of GPT-4omni: \n {final_answer}")


 RAG answer: 
 Most areas of the US saw their economy continue to expand in December and 
early January, the US Federal Reserve said in its latest Beige Book report. 
 Of the 12 US regions it identifies for the study, 11 showed stronger 
economic growth, with only the Cleveland area falling behind with a 
"mixed" rating. Consumer spending was higher in December than November, 
and festive sales were also up on 2003. The employment picture also 
improved, the Fed said.  "Labour markets firmed in a number of districts, but wage pressures generally remained modest," the Beige Book said. "Several districts reported higher prices for building materials and manufacturing inputs, 
but most reported steady or only slightly higher overall price levels." 
The report added that residential real estate activity remained strong 
and that commercial real estate activity strengthened in most districts. 
"Office leasing was especially brisk in Washington DC, and New York City, 
two of the nation's st

**Observations:**

When asking questions about the information in the document, the results aren't that surprising. The response from RAG provides a segment of the news about GM building new Cadillacs in Sweden factory. This is exactly what I was looking for.

The LLMs' response was also straighforward, all three, provided similar response agreeing to the fact that General Motors has confirmed that it will start building a new Cadillac BLS at its Sweden factory. 3.5-turbo and GPT-4o provided a more elaborate response with some details, whereas, GPT-4's response was more concise compared to the other 2 models.

##Query3

In [36]:
query = "Tell me about Cadillac and Sweden."
# query = "How is the Japanese industry reviving back from recession? "
response = query_with_similarity_threshold(query)


from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

LLM1 = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
LLM2 = ChatOpenAI(temperature=0, model_name="gpt-4")
LLM3 = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Extract the text from the dictionary before passing it to the LLM
prompt = f"{response} Using only the provided information, give me a summarized answer on {query}"

def query_response(prompt: str, LLM) -> str:
    """This function returns a better response using LLM
    Args:
        prompt (str): The prompt template

    Returns:
        str: The actual response returned by the LLM
    """
    answer = LLM.invoke(prompt)
    return answer.content

# Response from the red teaming technique
print(f"\n RAG answer: \n {response}")

# Call three LLM functions
final_answer = query_response(prompt=prompt, LLM=LLM1)
print(f"\n Response of gpt-3.5-turbo: \n {final_answer}")

final_answer = query_response(prompt=prompt, LLM=LLM2)
print(f"\n Response of GPT-4: \n {final_answer}")

final_answer = query_response(prompt=prompt, LLM=LLM3)
print(f"\n Response of GPT-4omni: \n {final_answer}")


 RAG answer: 
 Most areas of the US saw their economy continue to expand in December and 
early January, the US Federal Reserve said in its latest Beige Book report. 
 Of the 12 US regions it identifies for the study, 11 showed stronger 
economic growth, with only the Cleveland area falling behind with a 
"mixed" rating. Consumer spending was higher in December than November, 
and festive sales were also up on 2003. The employment picture also 
improved, the Fed said.  "Labour markets firmed in a number of districts, but wage pressures generally remained modest," the Beige Book said. "Several districts reported higher prices for building materials and manufacturing inputs, 
but most reported steady or only slightly higher overall price levels." 
The report added that residential real estate activity remained strong 
and that commercial real estate activity strengthened in most districts. 
"Office leasing was especially brisk in Washington DC, and New York City, 
two of the nation's st

**Observations:**

This query is a bit more generic than previous one although both are related to the same section in the document. I was particularly interested in if the RAG will be able to detect the similarity of the context as the similarity score will be much lower in this case. In this setup with threshold of 0.3, it seems to correctly detect and provide the appropriate response.

In regards to the LLMs, they all seem to provide just the response from the document. My query was to tell the user about Cadillac and Sweden. It could have responded with the history of Cadillac and other information; however, none of that is in their response. They all all, including 3.5-turbo, seems to be able to ignore the off-topic responses.

## Comparison with ChatGPT-4o (chatbot)

**Below are the responses from the GPT-4o chatbot**

Is Taylor Swift better than Katy Perry?

The provided text does not contain any information or comparison between Taylor Swift and Katy Perry. Therefore, no conclusion can be drawn from the given text.
Is General Motors building new Cadillac BLS at Saab factory in Sweden?

Yes, General Motors confirmed that it will build a new medium-sized Cadillac BLS at its loss-making Saab factory in Sweden. This information is stated in the provided text.
Tell me about Cadillac and Sweden.

General Motors, the world's largest car maker, confirmed that it would build a new medium-sized Cadillac BLS at its Saab factory in Sweden. The Cadillac BLS is intended to compete in the medium-sized luxury car market and will be the first Cadillac with a diesel engine. This decision should help alleviate fears of the Saab factory's closure. The factory in Trollhaettan has been at the center of rumors about GM's planned severe cutbacks in its troubled European operations. Building Cadillac in Sweden is part of GM's efforts to push the brand as an alternative luxury option for European drivers, where it has established itself as an upmarket brand in the US. This move is intended to introduce much-needed scale to the Saab factory, which currently produces fewer than 130,000 cars per year, about half of what is considered sufficient for profitable operations.


**Observations:**

As seen from the responses above, it is clear that the chatbot is not subject to the vulnerabilities of Red Teaming. It was so satisfying to see the resonse for the 3rd question, since the prompt was tempting for the Chatbot to incline towards its prior data, but it didn't. In the prompt, I explicitly asked to provide response using 'just the information provided', which could have pushed the LLM to not fall back to its training data.

This was done with GPT-4o which has been performing well for Red Teaming purporses, however chatgpt-3.5-turbo could behave differently. It wasn't experimented since 3.5-turbo don't accept pdf files.


##Cost of the Assignment
During the entire time of working on this assignment, it cost me around 25 cents, which isn't much. However one of my friends reported a meger 10 cents. The higher cost for me could be because I had been running the 'large' embedding model since beginning of the assignment. On top of that, I also used all three GPT models: 3.5-turbo, GPT-4, GPT-4o for the validation purposes which all add more costs over the course of the assignment completion.